In [1]:
library(magrittr)
library(DESeq2)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGe

In [ ]:
metaInfo <- read.table('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/intermediate/metaInfo.txt', header = T, sep = '\t')
head(metaInfo)

,Sample,Cancer,Group,DataBase,Group_
,<chr>,<chr>,<chr>,<chr>,<fct>
1,TCGA-FU-A3HZ-01,CESC,Tumor,TCGA,reference
2,TCGA-DR-A0ZM-01,CESC,Tumor,TCGA,reference
3,TCGA-IR-A3LB-01,CESC,Tumor,TCGA,reference
4,TCGA-DG-A2KJ-01,CESC,Tumor,TCGA,reference
5,TCGA-C5-A7CM-01,CESC,Tumor,TCGA,reference
6,TCGA-VS-A9UJ-01,CESC,Tumor,TCGA,reference


In [34]:
countData <- read.table('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/intermediate/luad_lusc_counts.txt',
    header = T, row.names = 1, sep = '\t', check.names = F) %>% as.matrix()
countData <- countData[grep('ENSG', x = rownames(countData)),]
head(countData)

,TCGA-FU-A3HZ-01,TCGA-DR-A0ZM-01,TCGA-IR-A3LB-01,TCGA-DG-A2KJ-01,TCGA-C5-A7CM-01,TCGA-VS-A9UJ-01,TCGA-VS-A8QH-01,TCGA-MY-A5BF-01,TCGA-JW-A5VJ-01,TCGA-C5-A1BN-01,⋯,TCGA-W5-AA38-01,TCGA-4G-AAZO-01,TCGA-ZD-A8I3-01,TCGA-W5-AA2O-01,TCGA-ZH-A8Y6-01,TCGA-W6-AA0S-01,TCGA-W5-AA33-01,TCGA-YR-A95A-01,TCGA-W5-AA2I-01,TCGA-W5-AA34-01
ENSG00000000003,3423,2162,10190,462,5349,2661,4561,1884,1582,1869,⋯,8249,1631,5254,2883,576,3632,11342,10086,4245,8150
ENSG00000000005,0,0,6,0,1,0,36,2,0,0,⋯,1,0,1,0,0,1,1,10,0,0
ENSG00000000460,459,339,912,47,494,273,337,834,603,1868,⋯,297,162,370,358,254,185,390,166,307,378
ENSG00000000938,210,719,323,91,177,74,115,350,401,81,⋯,182,137,143,248,103,570,668,151,118,223
ENSG00000001084,5721,820,3148,123,1406,979,2754,1240,615,3346,⋯,2055,447,2180,1266,2642,1136,2608,3206,1715,1853
ENSG00000001561,715,342,3520,102,2376,1611,1286,992,227,49,⋯,772,654,881,686,2245,816,1640,1052,613,1313


In [35]:
table(metaInfo['Sample'] == colnames(countData))


TRUE 
8232 

In [37]:
dim(countData)

[1] 14086  8232

In [59]:
genes <- read.table('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/select_genes/luad_lusc_unique_ensemble_ids.txt', sep = '\t', header = T)$EnsembleID
length(genes)

[1] 157

In [60]:
table(duplicated(genes))


FALSE 
  157 

In [61]:
metaInfo$Group_ <- ifelse(metaInfo$Cancer %in% c('LUSC', 'LUAD'), 'target', 'reference')
metaInfo$Group_ = factor(metaInfo$Group_, levels = c('target', 'reference'))
countData_ <- countData[genes,]
dim(countData_)

[1]  157 8232

In [62]:
table(metaInfo$Group_)


   target reference 
     1016      7216 

In [64]:
dim(metaInfo[metaInfo$Cancer %in% c('LUSC', 'LUAD'),])

[1] 1016    5

In [65]:
dds <- DESeqDataSetFromMatrix(
    countData = countData_,
    colData = metaInfo,
    design = ~ Group_
)
dds

class: DESeqDataSet 
dim: 157 8232 
metadata(1): version
assays(1): counts
rownames(157): ENSG00000032444 ENSG00000064999 ... ENSG00000280655
  ENSG00000281550
rowData names(0):
colnames(8232): TCGA-FU-A3HZ-01 TCGA-DR-A0ZM-01 ... TCGA-W5-AA2I-01
  TCGA-W5-AA34-01
colData names(5): Sample Cancer Group DataBase Group_

In [66]:
dim(countData_)

[1]  157 8232

In [67]:
smallestGroupSize <- .5
keep <- rowSums(counts(dds) > 0) / ncol(countData) >= smallestGroupSize
dds <- dds[keep,]

In [68]:
dds <- DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [69]:
res <- results(dds, contrast=c("Group_", 'target', 'reference'), tidy = T)
head(res)

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000032444,3562.25844,-0.27270447,0.02162072,-12.613107,1.788019e-36,7.549414e-36
2,ENSG00000064999,2548.50826,-0.07487034,0.02240760,-3.341292,8.338952e-04,1.022194e-03
3,ENSG00000070614,6227.43896,-0.48217248,0.02595000,-18.580830,4.593473e-77,5.818399e-76
4,ENSG00000080572,10.93696,1.49393982,0.12514264,11.937896,7.509848e-33,2.594311e-32
5,ENSG00000086544,2868.55934,0.35690400,0.03129331,11.405120,3.942332e-30,9.987241e-30
6,ENSG00000100442,2521.11338,-0.08202149,0.02153467,-3.808811,1.396364e-04,1.895065e-04


In [70]:
dim(res)

[1] 38  7

In [71]:
dim(dds@colData)

[1] 8232    7

In [72]:
readr::write_tsv(res, '/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/results/LUAD_LUSC/luad_lusc_unique.txt')